In [7]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

def black_scholes_call(S, K, t, r, sigma):
    """Calculates the Black-Scholes price for a European call option."""
    if t <= 0:
        return max(0, S - K)
    sigma_t_sqrt = sigma * np.sqrt(t)
    if sigma_t_sqrt == 0:
        return max(0, S * np.exp(-r * t) - K * np.exp(-r * t)) if S > K else 0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * t) / sigma_t_sqrt
    d2 = d1 - sigma_t_sqrt
    call_price = (S * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2))
    return call_price

# Custom function for MAPE to handle potential division by zero
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Avoid division by zero by filtering out zero values in the actual prices
    non_zero_mask = y_true != 0
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

# --- 1. Load the Data ---
df = pd.read_excel('Final_Without_IV_2.xlsx')

# --- 2. Prepare the Data ---
print("\nPreparing data for analysis...")
df['t_years'] = df['Time_period_t_in_days'] / 365.0
df['r_decimal'] = df['Risk_Free_Rate_r'] / 100.0
df['Moneyness'] = df['Underlying_Stock_Price_S'] / df['Strike_Price_K']
print("Data preparation complete.")

# --- 3. Calculate Error Metrics for Each Volatility Period ---
print("\n--- Calculating Error Metrics for each volatility period ---")

vol_columns = [
    'Historical_Volatility_20_days',
    'Historical_Volatility_30_days',
    'Historical_Volatility_40_days'
]

for vol_col in vol_columns:
    predicted_price_col = f'Predicted_Price_{vol_col[-7:]}'
    df[predicted_price_col] = df.apply(
        lambda row: black_scholes_call(
            S=row['Underlying_Stock_Price_S'],
            K=row['Strike_Price_K'],
            t=row['t_years'],
            r=row['r_decimal'],
            sigma=row[vol_col]
        ),
        axis=1
    )

    error_col = f'Error_{vol_col[-7:]}'
    df[error_col] = df['Option_Price_C'] - df[predicted_price_col]

    actual_prices = df['Option_Price_C']
    predicted_prices = df[predicted_price_col]

    # --- METRICS CALCULATION ---
    mse = mean_squared_error(actual_prices, predicted_prices)
    mae = mean_absolute_error(actual_prices, predicted_prices)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(actual_prices, predicted_prices)
    r2 = r2_score(actual_prices, predicted_prices)

    print(f"\n--- Results for {vol_col} ---")
    print(f"R-Squared (R²):                  {r2:.4f}")
    print(f"Mean Absolute Error (MAE):     {mae:,.2f}")
    print(f"Mean Abs. Percentage Err (MAPE): {mape:.2f}%")
    print(f"Mean Squared Error (MSE):      {mse:,.2f}")
    print(f"Root Mean Squared Error (RMSE):  {rmse:,.2f}")
    print("----------------------------------------------------------")




Preparing data for analysis...
Data preparation complete.

--- Calculating Error Metrics for each volatility period ---

--- Results for Historical_Volatility_20_days ---
R-Squared (R²):                  0.9915
Mean Absolute Error (MAE):     40.32
Mean Abs. Percentage Err (MAPE): 53.39%
Mean Squared Error (MSE):      6,398.98
Root Mean Squared Error (RMSE):  79.99
----------------------------------------------------------

--- Results for Historical_Volatility_30_days ---
R-Squared (R²):                  0.9931
Mean Absolute Error (MAE):     37.67
Mean Abs. Percentage Err (MAPE): 50.82%
Mean Squared Error (MSE):      5,180.33
Root Mean Squared Error (RMSE):  71.97
----------------------------------------------------------

--- Results for Historical_Volatility_40_days ---
R-Squared (R²):                  0.9937
Mean Absolute Error (MAE):     37.11
Mean Abs. Percentage Err (MAPE): 49.04%
Mean Squared Error (MSE):      4,749.44
Root Mean Squared Error (RMSE):  68.92
--------------------